# Cellpose : hyperparameter tuning

*NOTE : this notebook requires a separate environment with cellpose+jupyter installed.*

Here we check that cellpose hyperparameters are ideally set.

- Flow threshold : does not have an effect in 3D
- Probability threshold : range [-6;6], this should be verified
- Resample : check with/without
- Cell diameter : has already been set manually before training (should be optimal)

In [ ]:
# --------- REQUIRES A SEPARATE ENVIRONMENT WITH CELLPOSE INSTALLED --------- #
from cellpose import models 
from pathlib import Path
import numpy as np
from tifffile import imread, imwrite

: 

In [ ]:
DATA_PATH = Path.home() / "Desktop/Code/CELLSEG_BENCHMARK/RESULTS/SUPERVISED_PERF_FIG/"
pretrained_path = DATA_PATH / "weights/fold_1/cellpose/supervised_perf_fig.cellpose" 
image_path = DATA_PATH / "fold1/visual.tif"
image = imread(str(image_path))

In [ ]:
resample = [True, False]
prob_threshs = np.arange(-6, 7, 1) # from -6 to 6

model = models.CellposeModel(gpu=True, pretrained_model=str(pretrained_path), diam_mean=3.3)

In [ ]:
for resample_val in resample:
    for prob_t in prob_threshs:
        masks, flows, styles = model.eval(
            x=[image],
            do_3d=True,
            channels=[0,0],
            resample=resample_val,
            cellprob_threshold=prob_t,
        )
        save_name = f"./cellpose_masks/cellpose_masks_t_{prob_t}_r_{int(resample_val)}.tif"
        imwrite(save_name, masks)

In [ ]:
image_c5 = DATA_PATH / "fold2/c5image.tif"
image_c3 = DATA_PATH / "fold3/c3image.tif"
best_thresh = -6
resample_val = True

pretrained_c5 = DATA_PATH / "weights/fold_2/cellpose/fold2_cellpose.cellpose"
model_c5 = models.CellposeModel(gpu=True, pretrained_model=str(pretrained_path), diam_mean=3.3)
masks_c5, _, _ = model.eval(
    x=[imread(str(image_c5))],
    do_3d=True,
    channels=[0,0],
    resample=resample_val,
    cellprob_threshold=best_thresh,
)
imwrite("./cellpose_masks/cellpose_masks_c5.tif", masks_c5)


In [ ]:
pretrained_c3 = DATA_PATH / "weights/fold_3/cellpose/fold3_cellpose.cellpose"
masks_c3, _, _ = model.eval(
    x=[imread(str(image_c3))],
    do_3d=True,
    channels=[0,0],
    resample=resample_val,
    cellprob_threshold=best_thresh,
)

imwrite("./cellpose_masks/cellpose_masks_c3.tif", masks_c3)